In [7]:
import pipeline
from PIL import Image
import torch
from model_loader import load_models
import matplotlib.pyplot as plt
import numpy as np
from ddpm import DDPMSampler
from tqdm import tqdm
import torchvision

In [8]:
def get_time_embedding(timesteps: torch.LongTensor, dim: int = 160):
    if timesteps.dim() == 0:
        timesteps = timesteps.unsqueeze(0)
    device = timesteps.device
    half_dim = dim
    freqs = torch.pow(
        10000,
        -torch.arange(0, half_dim, dtype=torch.float32, device=device) / half_dim
    )
    args = timesteps.float().unsqueeze(1) * freqs.unsqueeze(0)
    emb = torch.cat([torch.cos(args), torch.sin(args)], dim=-1)
    return emb
def sample_i(h, w, vae, diffusion_model, generator, epoch, global_step, device, seed,sigma_latent,num_image):
    sampler_i = DDPMSampler(generator)
    sampler_i.set_inference_timesteps(1000)
    with torch.no_grad():
        latents = torch.randn((1, 4, h // 8, w // 8), device=device)
        for timestep in tqdm(sampler_i.timesteps, desc="Sampling"):
            t = torch.tensor([int(timestep)], dtype=torch.long, device=device)
            time_embedding = get_time_embedding(t).to(device)
            model_output = diffusion_model(latents, time_embedding)
            latents = sampler_i.step(timestep, latents, model_output)
        
        latents = latents*sigma_latent
        # Decode & log
        decoded = vae.decoder(latents)
        img = decoded.squeeze(0).cpu().numpy()
        img = np.transpose(img, (1, 2, 0))
        img = np.clip(img, 0.0, 1.0)
        img = (img * 255).astype(np.uint8)
        # convert to float in [0,1]
        tensor = torch.from_numpy(img).permute(2,0,1).float().div(255.0)
        torchvision.utils.save_image(tensor, f"outputSCALED_{seed}_{num_image}.png")
    #return img

In [9]:
from model_loader_conv import load_models
DEVICE = "cpu"

ALLOW_CUDA = True

if torch.cuda.is_available() and ALLOW_CUDA:
    DEVICE = "cuda"
diffusion,vae = load_models("checkpoints\diffusion/betaKL@1.0_conv_NU_constantLR_scaled/diffusion_best_epoch496_loss0.3401_1.pt",
                             "checkpoints/vae/betaKL@1.0/weights_ck_398.pt",
                             DEVICE
                             )
vae.eval()
diffusion.eval()
seed = 0
img_list = []
generator = torch.Generator(device=DEVICE)
generator.manual_seed(seed)
for image_num in range(8):
    sample_i(256, 256, vae, diffusion, generator, 0, 0, DEVICE, seed, 0.182473480701,image_num)
    #img_list.append(img_numpy)

Sampling: 100%|██████████| 1000/1000 [01:05<00:00, 15.18it/s]


In [ ]:
from model_loader_conv import load_models
DEVICE = "cpu"

ALLOW_CUDA = True
ALLOW_MPS = False

if torch.cuda.is_available() and ALLOW_CUDA:
    DEVICE = "cuda"
elif (torch.has_mps or torch.backends.mps.is_available()) and ALLOW_MPS:
    DEVICE = "mps"
print(f"Using device: {DEVICE}")

diffusion,vae = load_models("checkpoints/diffusion/betaKL@1.0_conv/diffusion_best_epoch30_loss0.1464.pt",
                             "checkpoints/vae/betaKL@1.0/weights_ck_398.pt",
                             DEVICE
                             )
# print the number of parameters in the model diffusion
print(f"Number of parameters in diffusion model: {sum(p.numel() for p in diffusion.parameters())}")
with torch.no_grad():
    vae.eval()
    diffusion.eval()


    models = {'diffusion':diffusion, 'vae':vae}
    ## SAMPLER

    sampler = "ddpm"
    num_inference_steps = 1000

    seeds = [40,50,60]
    for seed in seeds:
        img = pipeline.generate(
            sampler_name=sampler,
            n_inference_steps=num_inference_steps,
            seed=seed,
            models=models,
            device=DEVICE,
            idle_device="cpu"
        )
        # show it with PIL
        img = Image.fromarray(img)
        # savet it 
        img.save(f"output_conv_{seed}_double_loss.png")
    
 

Using device: cuda
Number of parameters in diffusion model: 592281604


100%|██████████| 1000/1000 [00:51<00:00, 19.47it/s]


In [ ]:
from model_loader_conv import load_models
DEVICE = "cpu"

ALLOW_CUDA = True
ALLOW_MPS = False

if torch.cuda.is_available() and ALLOW_CUDA:
    DEVICE = "cuda"
elif (torch.has_mps or torch.backends.mps.is_available()) and ALLOW_MPS:
    DEVICE = "mps"
print(f"Using device: {DEVICE}")

diffusion,vae = load_models("checkpoints/diffusion/betaKL@0.5_conv/diffusion_best_epoch14_loss0.0694.pt",
                             "checkpoints/vae/betaKL@0.5/weights_ck_398.pt",
                             DEVICE
                             )
# print the number of parameters in the model diffusion
print(f"Number of parameters in diffusion model: {sum(p.numel() for p in diffusion.parameters())}")
with torch.no_grad():
    vae.eval()
    diffusion.eval()


    models = {'diffusion':diffusion, 'vae':vae}
    ## SAMPLER

    sampler = "ddpm"
    num_inference_steps = 1000

    seed = 30
    
    img = pipeline.generate(
        sampler_name=sampler,
        n_inference_steps=num_inference_steps,
        seed=seed,
        models=models,
        device=DEVICE,
        idle_device="cpu"
    )
    # show it with PIL
    img = Image.fromarray(img)
    # savet it 
    img.save(f"output_conv_{seed}_0.5VAE.png")
    
 

Using device: cuda
Number of parameters in diffusion model: 592281604


100%|██████████| 1000/1000 [00:41<00:00, 24.12it/s]


In [ ]:
import pipeline
from PIL import Image
import torch
from model_loader import load_models
import matplotlib.pyplot as plt